In [1]:
import numpy as np

## Day 1

In [68]:
def DayOne_1(depth_readings):
    return np.sum([depth_readings[i] < depth_readings[i+1] for i in range(len(depth_readings)-1)])

depth=np.loadtxt("DayOne_input_1.txt",dtype='int')
smoothed_depth=[depth[i]+depth[i+1]+depth[i+2] for i in range(len(depth)-2)]

print("The number of times the deapth increased in successive scans: ",DayOne_1(depth))
print("The number of times the sum of measurements in the sliding window of length 3 increases: ",DayOne_1(smoothed_depth))

The number of times the deapth increased in successive scans:  1342
The number of times the sum of measurements in the sliding window of length 3 increases:  1378


## Day 2

In [3]:
horizontal_start,vertical_start=[0,0]
aim=0

def DayTwo_1(steps,starting_point):
    horiz=starting_point[0]
    vert=starting_point[1]
    for i in range(len(steps)):
        direction=steps[i][0][0]
        value=int(steps[i][-1])
        if direction=='f':
            horiz+=value
        elif direction=='u':
            vert-=value
        else:
            vert+=value
    return horiz*vert

def DayTwo_2(steps,starting_point,aim):
    horiz=starting_point[0]
    vert=starting_point[1]
    for i in range(len(steps)):
        direction=steps[i][0][0]
        value=int(steps[i][-1])
        if direction=='f':
            horiz+=value
            vert+=aim*value
        elif direction=='u':
            aim-=value
        else:
            aim+=value
    return horiz*vert


steps=np.loadtxt("DayTwo_input.txt",dtype='str')    
print("Product of final horizontal position with final depth: ", DayTwo_1(steps,[horizontal_start,vertical_start]))
print("Product of final horizontal position with final depth with updated correct directions: ", DayTwo_2(steps,[horizontal_start,vertical_start],aim))

Product of final horizontal position with final depth:  2102357
Product of final horizontal position with final depth with updated correct directions:  2101031224


In [60]:
horizontal_start,vertical_start=[0,0]
aim=0

def DayTwo_1(steps,starting_point):
    horiz=starting_point[0]
    vert=starting_point[1]
    for i in range(len(steps)):
        step=str(steps[i])
        value=int(float(step[-1]))
        if step.startswith("f"):
            horiz+=value
        elif step.startswith("u"):
            vert-=value
        else:
            vert+=value
    return horiz*vert

def DayTwo_2(steps,starting_point,aim):
    horiz=starting_point[0]
    vert=starting_point[1]
    for i in range(len(steps)):
        step=str(steps[i])
        value=int(step[-1])
        if step.startswith("f"):
            horiz+=value
            vert+=aim*value
        elif step.startswith("u"):
            aim-=value
        else:
            aim+=value
    return horiz*vert


steps=np.loadtxt("DayTwo_input.txt",dtype='str',delimiter="  ")    
print("Product of final horizontal position with final depth: ", DayTwo_1(steps,[horizontal_start,vertical_start]))
print("Product of final horizontal position with final depth with updated correct directions: ", DayTwo_2(steps,[horizontal_start,vertical_start],aim))

Product of final horizontal position with final depth:  2102357
Product of final horizontal position with final depth with updated correct directions:  2101031224


## Day 3

In [24]:
def power_consumption(readings):
    cutoff=readings.shape[1]/2
    summed_readings=np.sum(readings,axis=1)

    gamma_rate_bin="".join([str((x>cutoff)*1) for x in summed_readings])
    gamma_rate=int(gamma_rate_bin,2)
    epsilon_rate=int("".join(["1" if x=="0" else "0" for x in gamma_rate_bin]),2)
    return gamma_rate*epsilon_rate

readings=np.genfromtxt("DayThree_input.txt",dtype='int',delimiter=1,unpack=True)
print("Power consumption of the submarine (gamma rate * epsilon rate)= ",power_consumption(readings))

Power consumption of the submarine (gamma rate * epsilon rate)=  2583164


In [126]:
def common_bit(readings,length,mode="most",if_equally_common=0):
    cutoff=length/2
    summed_readings=np.sum(readings)
    
    if if_equally_common==0 and mode=="most":
        bit=(not summed_readings<=cutoff)*1
    elif if_equally_common==0 and mode=="least":
        bit=(not summed_readings>=cutoff)*1
    elif if_equally_common==1 and mode=="most":
        bit=(summed_readings>=cutoff)*1
    else:
        bit=(summed_readings<=cutoff)*1
    return bit

def life_support_rating(readings):
    
    x,y=readings.shape
    idx_O2=np.arange(y)
    idx_CO2=np.arange(y)
    run_O2=True; run_CO2=True
    
    for i in range(x):
        m=common_bit(readings[i,idx_O2], len(idx_O2), mode="most", if_equally_common=1)
        l=common_bit(readings[i,idx_CO2], len(idx_CO2), mode="least", if_equally_common=0)
        idx_O2_temp=[];idx_CO2_temp=[];
        for j in range(y):
            if run_O2 and j in idx_O2 and readings[i,j]==m:
                idx_O2_temp.append(j)
            if run_CO2 and j in idx_CO2 and readings[i,j]==l:
                idx_CO2_temp.append(j)
        idx_O2=idx_O2_temp;
        idx_CO2=idx_CO2_temp;
        if len(idx_O2)==1:
            O2=readings[:,idx_O2[0]] @ 2**np.arange(x)[::-1]
            run_O2=False;idx_O2=[]
        if len(idx_CO2)==1:
            CO2=readings[:,idx_CO2[0]] @ 2**np.arange(x)[::-1]
            run_CO2=False;idx_CO2=[]
        if len(idx_CO2)==1 and len(idx_O2)==1:
            break
        
    if len(idx_CO2)>1 or len(idx_CO2)>1:
        print("There are duplicate entries present in the readings. Modify algorithm to deal with the same!!!")
    return O2*CO2

readings=np.genfromtxt("DayThree_input.txt",dtype='int',delimiter=1,unpack=True)
print("Life_support_rating of the submarine (O2 generator rating * CO2 scrubbing rate)= ",life_support_rating(readings))

Life_support_rating of the submarine (O2 generator rating * CO2 scrubbing rate)=  2784375


# Day 4

In [439]:
from itertools import groupby

In [547]:
def check_win(board,mask,board_size):
    sum_1=board.sum(axis=0)
    sum_2=board.sum(axis=1)
    sum_mask=mask*board_size
    return (sum_1==sum_mask).any() or (sum_2==sum_mask).any()
    
def bingo_turn(boards,turn_value,mask,winning_board_nos,boards_won):
    win=False
    board_size=boards.shape[-1]
    turn_wins=[]
    for board_num,board in enumerate(boards):
        if board_num not in winning_board_nos:
            for index,value in np.ndenumerate(board):
                if value==turn_value:
                    value_=float(mask)
                    board[index]=value_
            win=check_win(board,mask,board_size)
            if win==True:
                boards_won+=1
                turn_wins.append(board_num)
                bingo_result=win
    return win, turn_wins
        
def best_board(boards,num_called,mask):
    
    boards_won=0
    winning_board_nos=[]
    winning_turn_values=[]
    for turn_value in num_called:
        if boards_won==boards.shape[0]:
            break
        bingo_result=bingo_turn(boards,turn_value,mask,winning_board_nos,boards_won) 
        if bingo_result[0]==True:
            winning_board_nos.extend(bingo_result[1])
            winning_turn_values.append(turn_value)  
    first_win_value=winning_turn_values[0]
    best_board_num=winning_board_nos[0]#............I am effectively ignoring the fact that there can be multiple victor bords in each turns in this step
    best_board=boards[best_board_num,:,:]
    best_score= int(np.sum(best_board))*first_win_value
    
    last_win_value=winning_turn_values[-1]
    last_board_num=winning_board_nos[-1]#............I am effectively ignoring the fact that there can be multiple victor bords in each turns in this step
    last_board=boards[last_board_num,:,:]
    last_score=int(np.sum(last_board))*last_win_value
    
    return best_score,best_board_num,last_score,last_board_num

# Read in the call values and the bingo boards from input file.

num_called=np.genfromtxt("DayFour_input.txt",dtype='int',delimiter=',',unpack=True,max_rows=1)
array_list = []
with open('DayFour_input.txt') as f_data:  
    next(f_data)
    for k, g in groupby(f_data, lambda x: x.startswith('\n')):
        if not k:
            array_list.append(np.array([[float(x) for x in d.split()] for d in g if len(d.strip())]))#......len(d.strip()) for line '\n' is 0 -> False
boards=np.array(array_list).astype('float')

# Execute the bingo game and get the best/worst board and its score.

mask=0.01

best_score,best_board_num,last_score,last_board_num=best_board(boards,num_called,mask)
print("The board_{} (index {}) has the best chance of winning with score {}.".format(best_board_num+1,best_board_num,best_score))
print("The board_{} (index {}) has the last chance of winning with score {}.".format(last_board_num+1,last_board_num,last_score))

The board_3 (index 2) has the best chance of winning with score 7370.
The board_10 (index 9) has the last chance of winning with score 23670.


### Use zip() to merge to gether the value called and winning board numbers. This is to be able to output them both.

In [547]:
def check_win(board,mask,board_size):
    sum_1=board.sum(axis=0)
    sum_2=board.sum(axis=1)
    sum_mask=mask*board_size
    return (sum_1==sum_mask).any() or (sum_2==sum_mask).any()
    
def bingo_turn(boards,turn_value,mask,winning_board_nos,boards_won):
    win=False
    board_size=boards.shape[-1]
    turn_wins=[]
    for board_num,board in enumerate(boards):
        if board_num not in winning_board_nos:
            for index,value in np.ndenumerate(board):
                if value==turn_value:
                    value_=float(mask)
                    board[index]=value_
            win=check_win(board,mask,board_size)
            if win==True:
                boards_won+=1
                turn_wins.append(board_num)
                bingo_result=win
    return win, turn_wins
        
def best_board(boards,num_called,mask):
    
    boards_won=0
    winning_board_nos=[]
    winning_turn_values=[]
    for turn_value in num_called:
        if boards_won==boards.shape[0]:
            break
        bingo_result=bingo_turn(boards,turn_value,mask,winning_board_nos,boards_won) 
        if bingo_result[0]==True:
            winning_board_nos.extend(bingo_result[1])
            winning_turn_values.append(turn_value)  
    first_win_value=winning_turn_values[0]
    best_board_num=winning_board_nos[0]#............I am effectively ignoring the fact that there can be multiple victor bords in each turns in this step
    best_board=boards[best_board_num,:,:]
    best_score= int(np.sum(best_board))*first_win_value
    
    last_win_value=winning_turn_values[-1]
    last_board_num=winning_board_nos[-1]#............I am effectively ignoring the fact that there can be multiple victor bords in each turns in this step
    last_board=boards[last_board_num,:,:]
    last_score=int(np.sum(last_board))*last_win_value
    
    return best_score,best_board_num,last_score,last_board_num

# Read in the call values and the bingo boards from input file.

num_called=np.genfromtxt("DayFour_input.txt",dtype='int',delimiter=',',unpack=True,max_rows=1)
array_list = []
with open('DayFour_input.txt') as f_data:  
    next(f_data)
    for k, g in groupby(f_data, lambda x: x.startswith('\n')):
        if not k:
            array_list.append(np.array([[float(x) for x in d.split()] for d in g if len(d.strip())]))#......len(d.strip()) for line '\n' is 0 -> False
boards=np.array(array_list).astype('float')

# Execute the bingo game and get the best/worst board and its score.

mask=0.01

best_score,best_board_num,last_score,last_board_num=best_board(boards,num_called,mask)
print("The board_{} (index {}) has the best chance of winning with score {}.".format(best_board_num+1,best_board_num,best_score))
print("The board_{} (index {}) has the last chance of winning with score {}.".format(last_board_num+1,last_board_num,last_score))

The board_3 (index 2) has the best chance of winning with score 7370.
The board_10 (index 9) has the last chance of winning with score 23670.
